In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [2]:
SQUARE_SIZE = 512

In [3]:
# Funções de processamento de imagem

def adjust_exposure(image, gamma=1.0):
    invGamma = 21.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

def increase_brightness_contrast(image, alpha=1.0, beta=0):
    new_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return new_image

def enhance_color(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:, :, 1] = cv2.multiply(hsv[:, :, 1], 1.5)
    enhanced_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return enhanced_image


def apply_clahe(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=0.1, tileGridSize=(8, 8))
    clahe_applied = clahe.apply(gray)
    clahe_applied = cv2.cvtColor(clahe_applied, cv2.COLOR_GRAY2BGR)
    return clahe_applied

def remove_noise_rbg(image):
    return cv2.fastNlMeansDenoisingColored(image, None, 3, 3, 7, 21)

def remove_granular_noise(image):
    denoised = cv2.bilateralFilter(image, 15, 75, 75) 
    return denoised

def highlight_edges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    highlighted = cv2.addWeighted(image, 1.2, edges_colored, 0.2, 0)
    return highlighted



def process_square(square):
    color_image = cv2.cvtColor(square, cv2.COLOR_RGBA2BGR)
    bright_contrast = increase_brightness_contrast(color_image, alpha=-12, beta=50)
    adjusted = adjust_exposure(bright_contrast, 30)
    denoised = remove_noise_rbg(adjusted)
    denoised_granular = remove_granular_noise(denoised)
    enhanced = enhance_color(denoised_granular)
    highlighted = highlight_edges(enhanced)
    return highlighted


In [4]:
# Função para segmentação baseada nas regras propostas
def segment_image(image):
    # Segmentação baseada no canal vermelho da imagem
    red_channel = image[:, :, 2]
    _, segmented_image = cv2.threshold(red_channel, 100, 255, cv2.THRESH_BINARY)
    return segmented_image

# Função de filtragem por tamanho (remover objetos menores que 30 pixels)
def filter_by_size(image, min_size=30):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_image = np.zeros_like(image)
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area >= min_size:
            cv2.drawContours(filtered_image, [contour], -1, 255, thickness=cv2.FILLED)
    
    return filtered_image



In [5]:
# Funções de detecção e contagem de objetos vermelhos

def detect_red_objects(square):
    """Detecta objetos na faixa de cor vermelha em um quadrado da imagem no espaço RGB."""
    # Definir intervalos para a cor vermelha no espaço RGB
    lower_red1 = np.array([100, 0, 0])  # Vermelho mais escuro
    upper_red1 = np.array([255, 80, 80])  # Vermelho mais claro
    
    # Criar uma máscara para o intervalo de vermelho
    mask = cv2.inRange(square, lower_red1, upper_red1)
    
    # Aplicar a máscara na imagem original
    red_objects = cv2.bitwise_and(square, square, mask=mask)
    
    return red_objects, mask

def count_objects_in_red_range(square, plot=False):
    """Conta o número de objetos vermelhos em um quadrado da imagem e ignora objetos menores que 50 pixels."""
    red_objects, mask = detect_red_objects(square)
    
    # Encontrar contornos (objetos) na máscara binarizada
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contour_img = square.copy()
    cv2.drawContours(contour_img, contours, -1, (0, 255, 0), 2)

    if plot:
        # Plotagem
        fig, ax = plt.subplots(1, 3, figsize=(18, 6))
        ax[0].imshow(square)
        ax[0].set_title("Imagem Original")
        ax[1].imshow(mask, cmap='gray')
        ax[1].set_title("Máscara Vermelha")
        ax[2].imshow(contour_img)
        ax[2].set_title("Contornos Detectados")
        plt.show()

    return len(contours), contour_img


In [6]:

# Funções de processamento de imagem por quadrados

def divide_and_process_image(image):
    img_height, img_width, _ = image.shape
    total_objects = 0
    processed_squares = []
    no_processed_squares = []

    for y in range(0, img_height, SQUARE_SIZE):
        for x in range(0, img_width, SQUARE_SIZE):
            square = image[y:y + SQUARE_SIZE, x:x + SQUARE_SIZE]
            processed_square = process_square(square)
            no_processed_squares.append({"square": square})
            num_objects, contours_img = count_objects_in_red_range(processed_square)
            processed_squares.append({"square": processed_square, "contours_img": contours_img, "num_objects": num_objects})
            total_objects += num_objects

    return processed_squares, no_processed_squares, img_height, img_width, total_objects

def reconstruct_image(squares, img_height, img_width):
    reconstructed_image = np.zeros((img_height, img_width, 3), dtype=np.uint8)

    count = 0
    for y in range(0, img_height, SQUARE_SIZE):
        for x in range(0, img_width, SQUARE_SIZE):
            square = squares[count]["square"]
            h, w, _ = square.shape
            padded_square = np.zeros((SQUARE_SIZE, SQUARE_SIZE, 3), dtype=np.uint8)
            padded_square[:h, :w] = square # Preenche o quadrado com a imagem processada

            reconstructed_image[y:y + h, x:x + w] = padded_square[:h, :w]
            count += 1

    return reconstructed_image

In [7]:
# Execução do pipeline

image_path = '/media/williancarddd/NVME/fiotec/AETrampa/aedes_eggs_data/images/172-1.jpeg'

# resize 4k image

image = cv2.imread(image_path)
image = cv2.resize(image, (3840, 2160))

processed_squares, no_processed_squares, img_height, img_width, total_objects = divide_and_process_image(image)
reconstructed_image = reconstruct_image(processed_squares, img_height, img_width)

# Salvar a imagem reconstruída
cv2.imwrite('reconstructed_image.jpg', reconstructed_image)

print(f"Total de objetos vermelhos na imagem: {total_objects}")


[ WARN:0@0.310] global loadsave.cpp:241 findDecoder imread_('/media/williancarddd/NVME/fiotec/AETrampa/aedes_eggs_data/images/172-1.jpeg'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /croot/opencv-suite_1722029125240/work/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
# plot squadre with most objects
max_objects = 0
max_index = 0
for i, square in enumerate(processed_squares):
    if square["num_objects"] > max_objects:
        max_objects = square["num_objects"]
        max_index = i

square = processed_squares[1]["square"]
contours_img = processed_squares[max_index]["contours_img"]

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(square)
ax[0].set_title("Quadrado com mais objetos")
ax[1].imshow(contours_img)
ax[1].set_title("Contornos detectados")
plt.show()


In [ ]:
# Plotar a imagem original e a reconstruída
fig, ax = plt.subplots(1, 2, figsize=(25, 10))
ax[0].imshow(reconstructed_image)
ax[0].set_title("Imagem Reconstruída")
ax[1].imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
ax[1].set_title("Imagem Original")
plt.show()